# Session 06 - Reciprocal Best BLAST Hits (RBBH) <img src="data/JHI_STRAP_Web.png" style="width: 150px; float: right;"> 

## Learning Outcomes

* Conduct `BLASTP` comparisons between protein complements for prokaryotes
* Identify reciprocal best `BLAST` matches
* Visualise and interpret reciprocal best `BLAST` matches using a range of tools

## Introduction

### Reciprocal best `BLAST` hits

As discussed in the lecture, reciprocal Best `BLAST` Hits (RBBH) are a common approximation to orthology for protein sequences, used in comparative genomics. An RBBH is found when two proteins, each in a different genome, find each other as the best scoring match in the other genome.

### Python code

We will use the [`Biopython`](http://www.biopython.org) libraries to interact with and manipulate sequence data, and the [`Pandas`](http://pandas.pydata.org/) data analysis libraries to manipulate numerical data and `BLAST` output.

Most of the code you will use is imported from the local `bs32010` module in this directory, to avoid clutter in this notebook. You can inspect this module if you are interested in how the functions work.

In [ ]:
%pylab inline

from bs32010 import ex06  # Local functions and data

## 1. One-way `BLASTP` comparisons for RBBH

### Two organisms

Reciprocal best hits are built from "one-way" best hits. For organisms `A` and `B`, the best hits to the proteins of `A` in `B` would be found by a one-way `BLASTP` search of the proteins of `A` against those of `B`. Likewise, the best hits to the proteins of `B` in `A` would be found by a one-way `BLASTP` search of the proteins of `B` against those of `A`. This would give us two `BLAST` output files:

* `A_vs_B.tab`
* `B_vs_A.tab`

Then, to identify the RBBHs for `A` and `B`, we would identify those proteins of `A` and `B` that find each other as the best-scoring match in both files.

### Three or more organisms

The number of `BLASTP` comparisons we need to perform increases quickly with the number of additional organisms we include. For three organisms (`X`, `Y`, `Z`), we aim to produce three sets of RBBH, for each of the ways of combining the organisms in pairs:

* `X` vs `Y`
* `X` vs `Z`
* `Y` vs `Z`

and we would need to run two one-way `BLASTP` searches for each of those outputs.

For $k$ organisms, we need to run $k^2 - k = k(k -1)$ `BLASTP` searches, in order to obtain $\frac{k(k -1)}{2}$ RBBH output files. This is, for even reasonably small numbers of organisms, time-consuming and best automated by a script.

There are alternative, faster, sequence alignment/search programs that could be used, but the choice of program affects the final results, as found in [Ward and Moreno-Hagelsieb (2014)](http://dx.doi.org/10.1371/journal.pone.0101850).

### Coverage and identity

BLAST matches provide two parameters by which we can rank and filter results: *coverage* and *identity*.

* **identity**: the percentage of the BLASTP alignment that is made up of identical amino acid sequence, in the two matching sequences.
* **coverage**: There are two coverage values, one for the query sequence, and one for the subject sequence. They represent the percentage of the query (or subject) sequence that is covered by the aligned region.

These are useful parameters because they allow us to tune our `BLAST` output to reflect the degree of sequence similarity (and therefore presumed functional similarity) we want to have in our final RBBH set.

To calculate percentage identity ($PID$) of the `BLAST` match directly (though this value is provided for us to use in the `BLASTP` output), we would calculate:

$$
PID = \frac{\text{identical_sites}}{\text{alignment_length}}
$$

and to calculate query and subject coverage ($COV_{q}$ and $COV_{s}$, which we can request from custom `BLASTP` output), we find:

$$
COV_q = \frac{\text{alignment_length}}{\text{query_length}} \\
COV_s = \frac{\text{alignment_length}}{\text{subject_length}}
$$

### Sequence and `BLAST` output data

You will be working with the chromosomal protein complements of related bacteria are provided in the `rbbh_data` directory:

* `NC_004547.faa`: *Pectobacterium atrosepticum* SCRI1043
* `NC_013421.faa`: *Pectobacterium wasabiae* WPP163
* `NC_010694.faa`: *Erwinia tasmaniensis* ET1_99

and BLASTP comparisons of each protein sequence file that, to save time (this takes ≈15min on my laptop, without parallelisation), have already been conducted against both of the other protein sequence files:

* `NC_004547` vs `NC_013421`
* `NC_004547` vs `NC_010694`
* `NC_013421` vs `NC_004547`
* `NC_013421` vs `NC_010694`
* `NC_010694` vs `NC_004547`
* `NC_010694` vs `NC_013421`

using BLAST commands of the form:

```
blastp -query data/NC_004547.faa -subject data/NC_013421.faa \
       -outfmt "6 qseqid sseqid qlen slen length nident pident evalue bitscore" \
       -out output/NC_004547_vs_NC_013421.tab -max_target_seqs 1
```

If you would like to see how they were run, please examine the accompanying script file `run_rbbh_blast.sh`

The one-way `BLAST` output is found in the `rbbh_data/rbbh_output` directory:

```bash
rbbh_data/
├── NC_004547.faa
├── NC_004547.gbk
├── NC_010694.faa
├── NC_010694.gbk
├── NC_013421.faa
├── NC_013421.gbk
└── rbbh_output
    ├── NC_004547.faa.phr
    ├── NC_004547.faa.pin
    ├── NC_004547.faa.psq
    ├── NC_004547_vs_NC_010694.tab
    ├── NC_004547_vs_NC_013421.tab
    ├── NC_010694.faa.phr
    ├── NC_010694.faa.pin
    ├── NC_010694.faa.psq
    ├── NC_010694_vs_NC_004547.tab
    ├── NC_010694_vs_NC_013421.tab
    ├── NC_013421.faa.phr
    ├── NC_013421.faa.pin
    ├── NC_013421.faa.psq
    ├── NC_013421_vs_NC_004547.tab
    └── NC_013421_vs_NC_010694.tab
```

** Exercise 1 (5min): Inspect the `rbbh_data/rbbh_output` to see the one-way `BLASTP` output **

## 2. Parsing `BLASTP` output

The `BLASTP` output in `rbbh_data/rbbh_output` is in plain text, tab-separated format. `BLASTP` was run set to find only a single match for each protein query, and the output is not in standard `BLASTP` form. The columns in each of the `*.tab` files correspond to:

`query_id, subject_id, query_length, subject_length, alignment_length, identical_sites, percentage_identity, E-value , bitscore`

The function `ex06.find_rbbh()` will find the RBBH for any pair of the three files `NC_004547`, `NC_013421`, and `NC_010694`. It returns three `Pandas` dataframes:

* `df1`: the forward direction one-way BLAST search data
* `df2`: the reverse direction one-way BLAST search data
* `rbbh`: the corresponding set of reciprocal best hits

For example, the code below shows the first few lines of each of the dataframes for the comparison between SCRI1043 and *Erwinia tasmaniensis*.

```python
df1, df2, rbbh1 = ex06.find_rbbh('NC_004547', 'NC_010694')
df1.head()
df2.head()
rbbh1.head()
```

** Exercise 2(5min): Run this code, and inspect the first few lines of each dataframe **

In [ ]:
# Enter code here
df1, df2, rbbh1 = ex06.find_rbbh('NC_004547', 'NC_010694')
df1.head()
df2.head()
rbbh1.head()

** Exercise 3 (5min): Perform the same analysis for the *Pectobacterium* chromosomes `NC_004547` and `NC_013421`, using the variable names `df3`, `df4`, and `rbbh2` **

In [ ]:
# Enter code here
df3, df4, rbbh2 = ex06.find_rbbh('NC_004547', 'NC_013421')
df3.head()
df4.head()
rbbh2.head()

You can find the number of rows in a dataframe using the function `len()`. For example, the dataframe `df1` should have 4466 rows, if you run the function `len(df1)`.

** Exercise 4 (5min): How many rows do the one-way hits for each analysis have, and how many RBBH result in each analysis? **

* What proportion of one-way hits is lost in each analysis?
* Which comparison produces the most RBBH?
* Why are the two results different?

In [ ]:
# Enter code here
len(df1), len(df2), len(rbbh1), len(df3), len(df4), len(rbbh2)

By using the `.describe()` *method* of a dataframe, we can get more summary statistical information, such as the means and quantiles of a column of data. For example, to get a summary of the information in `rbbh1` we could run

```python
rbbh1.describe()
```

** Exercise 5 (5min): Use the `.describe()` method for both RBBH results, and look at the data for percentage identity and percentage coverage. **

* How do these values differ between the two sets of results?
* Why do you think they differ in this way?
* Why do you think the maximum coverage can exceed 100%?

In [ ]:
# Enter code here
rbbh1.describe()
rbbh2.describe()